# Практическое задание к уроку 7

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.1.0
Eager mode:  True
Hub version:  0.10.0
GPU is available


#### 1. Поэкспериментировать с моделью генерации текста, попробовать модель на словах, посмотреть что отработает лучше

In [4]:
path_to_file = 'evgenyi_onegin.txt'
path_to_file_1 = 'War_1.txt'

In [5]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [6]:
text_1 = open(path_to_file_1, 'rb').read().decode(encoding='utf-8')

In [7]:
#text=text_1#[:586544]

In [8]:
print(text[300:600])

   Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высоких дум и простоты;
                        Но так и быть - рукой пристрастной
                     


In [9]:
vocab_char = sorted(set(text))
print('{} unique characters'.format(len(vocab_char)))

131 unique characters


In [10]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
nltk.download("punkt")



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Cats\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [18]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_sentence(w):
  w = w.lower().strip()

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w



In [50]:
tokens = word_tokenize(preprocess_sentence(text))
vocab_word = sorted(set(tokens))
print('{} unique characters'.format(len(vocab_word)))


8501 unique characters


In [51]:
char2idx = {u:i for i, u in enumerate(vocab_word)}
idx2char = np.array(vocab_word)

text_as_int_word = np.array([char2idx[c] for c in tokens])

In [52]:
# The maximum length sentence you want for a single input in characters
seq_length = 6
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
word_dataset = tf.data.Dataset.from_tensor_slices(text_as_int_word)

In [53]:
for i in  word_dataset.take(1):
    print(i)

tf.Tensor(3, shape=(), dtype=int32)


In [54]:
text_as_int_word

array([ 3, 98,  4, ...,  3, 32,  4])

In [55]:
print(text[:1000])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высоких дум и простоты;
                        Но так и быть - рукой пристрастной
                        Прими собранье пестрых глав,
                        Полусмешных, полупечальных,
                        Простонародных, идеальных,
                        Небрежный плод моих забав,
                        Бессонниц, легких вдохновений,
                        Незрелых и увядших лет,
                        Ума холодных наблюдений
                        И сердца горестных замет.

            

In [56]:
text_as_int_word
idx2char[229]

'багряною'

In [57]:
sequences = word_dataset.batch(seq_length+1, drop_remainder=True)

In [58]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
   
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [59]:
for i in  dataset.take(1):
    print(i[0])
    

tf.Tensor([   3   98    4  184 6528 5923], shape=(6,), dtype=int32)


In [60]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 6), (64, 6)), types: (tf.int32, tf.int32)>

In [61]:
# Length of the vocabulary in chars
vocab_size = len(vocab_word)

# The embedding dimension
embedding_dim = 300

# Number of RNN units
rnn_units = 512

In [62]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [63]:
model = build_model(
    vocab_size=len(vocab_word),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [64]:
import os
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    period=20,
    save_weights_only=True)

In [65]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model.compile(optimizer='adam', loss=loss)

In [75]:
EPOCHS = 200
history = model.fit(dataset, epochs=EPOCHS,callbacks=[checkpoint_callback])

Train for 64 steps
Epoch 1/200
64/64 [==============================] - 5s 78ms/step - loss: 7.3712
Epoch 2/200
64/64 [==============================] - 5s 76ms/step - loss: 7.3709
Epoch 3/200
64/64 [==============================] - 5s 78ms/step - loss: 7.3705
Epoch 4/200
64/64 [==============================] - 5s 79ms/step - loss: 7.3746
Epoch 5/200
64/64 [==============================] - 5s 79ms/step - loss: 7.3699
Epoch 6/200
64/64 [==============================] - 5s 77ms/step - loss: 7.3710
Epoch 7/200
64/64 [==============================] - 5s 75ms/step - loss: 7.3689
Epoch 8/200
64/64 [==============================] - 5s 76ms/step - loss: 7.3613
Epoch 9/200
64/64 [==============================] - 5s 74ms/step - loss: 7.3695
Epoch 10/200
64/64 [==============================] - 5s 74ms/step - loss: 7.3720
Epoch 11/200
64/64 [==============================] - 5s 75ms/step - loss: 7.3604
Epoch 12/200
64/64 [==============================] - 5s 75ms/step - loss: 7.3677
Epoch 

64/64 [==============================] - 5s 76ms/step - loss: 7.3716
Epoch 199/200
64/64 [==============================] - 5s 74ms/step - loss: 7.3729
Epoch 200/200
64/64 [==============================] - 6s 92ms/step - loss: 7.3679


In [76]:
model_pr = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model_pr.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model_pr.build(tf.TensorShape([1, None]))

In [78]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate =30

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in word_tokenize(start_string)]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
   # temperature = 1.8

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        
        # using a categorical distribution to predict the character returned by the model
      #  predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
      #  print(idx2char[predicted_id])
        text_generated.append(idx2char[predicted_id])

    return (start_string + ' '.join(text_generated))

start_string=u"онегин "
generate_text(model_pr, start_string)

'онегин полны вперед бывало олигархических признанья ним чепец кони больше но сам глядела близ женой . пар быстрой пиров вестью . первоначальные счастья вы еще снеге , мужчин кто найдено жизнь'

'Онегин переплывал Улан явилась XXXI ... , Наедине чтоб , что , он , Как молодой ! Онегин Явились от знает враньем ; , . , все падут Тогда места : . бы Я трепещет кабинет Вдруг вкусу . . Иль . мечтой Лондон тошен искренней Брожу не к кладбище чтоб

#### по символьная модель может генерировать новые слова, в то время как словесная только слова определенные в тексте обучения. по символьная модель отработала лучше

#### 2. Поэкспериментировать с переводом написать по вашим наблюдениям где он ошибается, попробовать изменить архитектуру

In [29]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

In [30]:
path_to_file = "rus.txt"

In [31]:
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

    return zip(*word_pairs)

In [32]:
en, ru = create_dataset(path_to_file, None)
print(en[-1])
print(ru[-1])

<start> doubtless there exists in this world precisely the right woman for any given man to marry and vice versa but when you consider that a human being has the opportunity of being acquainted with only a few hundred people , and out of the few hundred that there are but a dozen or less whom he knows intimately , and out of the dozen , one or two friends at most , it will easily be seen , when we remember the number of millions who inhabit this world , that probably , since the earth was created , the right man has never yet met the right woman . <end>
<start> несомненно , для каждого мужчины в этом мире где то есть подходящая женщина , которая может стать ему женой , обратное верно и для женщин . но если учесть , что у человека может быть максимум несколько сотен знакомых , из которых лишь дюжина , а то и меньше , тех , кого он знает близко , а из этой дюжины у него один или от силы два друга , то можно легко увидеть , что с уч том миллионов живущих на земле людей , ни один подходящи

In [33]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
          filters='')
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

    return tensor, lang_tokenizer

def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
    targ_lang, inp_lang = create_dataset(path, num_examples)
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [34]:
num_examples = 100000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

80000 80000 20000 20000


In [35]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 300
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [36]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=False,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [37]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    # self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden):
    # enc_output shape == (batch_size, max_length, hidden_size)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)

    # passing the concatenated vector to the GRU
    output, state = self.gru(x, initial_state=hidden)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)

    return x, state

In [39]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)


In [40]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

decoder_sample_x, decoder_sample_h = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden)
decoder_sample_h.shape

TensorShape([64, 1024])

In [41]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [42]:
checkpoint_dir = './training_nmt_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [35]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden = decoder(dec_input, dec_hidden)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [37]:
EPOCHS = 50

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

  #  if batch % 100 == 0:
  #    print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
  #                                                 batch,
  #                                                 batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Loss 0.7010
Time taken for 1 epoch 274.1226465702057 sec

Epoch 2 Loss 0.3762
Time taken for 1 epoch 296.9289231300354 sec

Epoch 3 Loss 0.2238
Time taken for 1 epoch 295.0496287345886 sec

Epoch 4 Loss 0.1525
Time taken for 1 epoch 274.4653480052948 sec

Epoch 5 Loss 0.1189
Time taken for 1 epoch 271.6058373451233 sec

Epoch 6 Loss 0.1015
Time taken for 1 epoch 274.47633934020996 sec

Epoch 7 Loss 0.0915
Time taken for 1 epoch 272.8687379360199 sec

Epoch 8 Loss 0.0862
Time taken for 1 epoch 272.1854729652405 sec

Epoch 9 Loss 0.0815
Time taken for 1 epoch 269.4956681728363 sec

Epoch 10 Loss 0.0790
Time taken for 1 epoch 272.57602858543396 sec

Epoch 11 Loss 0.0758
Time taken for 1 epoch 271.0193099975586 sec

Epoch 12 Loss 0.0736
Time taken for 1 epoch 274.3884150981903 sec

Epoch 13 Loss 0.0721
Time taken for 1 epoch 270.2729959487915 sec

Epoch 14 Loss 0.0708
Time taken for 1 epoch 274.0317270755768 sec

Epoch 15 Loss 0.0680
Time taken for 1 epoch 270.3639144897461 sec

Ep

#### Перевод

In [3]:
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden = decoder(dec_input, dec_hidden)

    # storing the attention weights to plot later on
    predicted_id = tf.argmax(predictions[0]).numpy()
    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence

In [4]:
def translate(sentence):
  result, sentence = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

In [43]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [55]:
translate('Я не смогу поехать.')

Input: <start> я не смогу поехать . <end>
Predicted translation: i can't go . <end> 


In [57]:
translate('Со следующей недели начинаются экзамены. В последнюю ночь тебе не поможет, там будет много вопросов. Тебе нужно начать заниматься сегодня')

Input: <start> со следующей недели начинаются экзамены . в последнюю ночь тебе не поможет , там будет много вопросов . тебе нужно начать заниматься сегодня <end>
Predicted translation: you should be home ? <end> 


In [58]:
translate('Со следующей недели начинаются экзамены')

Input: <start> со следующей недели начинаются экзамены <end>
Predicted translation: next child am in peace . <end> 


#### по тесту перевода модель все таки недоучилась + надо больше примеров. В целом самой модели не хватает контекстности. 